In [1]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install insta-scrape

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd 

In [ ]:
#scraping profile 
#scraper94118 = fake profile /username 

#to scrape posts and hashtag 
#to_dict
#to_json
#to_csv

In [6]:

from instascrape import Profile 
profile = Profile('scraper94118')
profile.scrape()
recent_posts = profile.get_recent_posts()

In [ ]:
#visulatising 

In [9]:
posts_data = [post.to_dict() for post in recent_posts]
posts_df = pd.DataFrame(posts_data)
print(posts_df[['upload_date', 'comments', 'likes']])

NameError: name 'recent_posts' is not defined

In [10]:
import matplotlib.pyplot as plt 

plt.style.use('seaborn-darkgrid')      # Stylistic change

plt.scatter(df.upload_date, df.likes)  # Plot the data
plt.xlabel('Upload Date')              # Write labels
plt.ylabel('Likes')
plt.title('@chris_greening Likes per Post')
plt.show()                             # Show graph 

NameError: name 'df' is not defined

In [ ]:
#tracking 

In [ ]:
import time 
import datetime 

import instascrape

def track_post(url: str):
    """
    Return a list of datetimes and a list of strings from an 
    Instagram post scraped across a one hour period.
    """ 
    times = [] 
    likes = []
    now = datetime.datetime.now()
    end_time = now + datetime.timedelta(hours=1)

    while now < end_time:
        time.sleep(60)     

        post = instascrape.Post(url) 
        post.scrape()

        now = datetime.datetime.now()

        times.append(now)
        likes.append(post.likes)
    return times, likes

In [ ]:
#location scraping 

In [ ]:
from instascrape import Location 
url = 'https://www.instagram.com/explore/locations/212988663/new-york-new-york/'
new_york = Location(url)
new_york.scrape()

In [ ]:
#example 

In [ ]:
print(f"The NY location tag has {new_york.amount_of_posts:,} posts")
>>> The NY location tag has 61,202,403 posts.

print(f"NY tag geographic coordinates: ({new_york.latitude}, {new_york.longitude}")
>>> NY tag geographic coordinates: (40.7142, -74.0064)

In [ ]:
#second method that i've foudn 


In [ ]:
scrapy startproject instascraper

cd instascraper

scrapy genspider instagram instagram.com

In [ ]:
#get five functions from this 
#start_requests - will construct the Instagram URL for the users account and send the request to Instagram.
#parse - will extract all the posts data from the users news feed.
#parse_page - if there is more than one page, this function will parse all the posts data from those pages.
#get_video - if the post includes a video, this function will be called and extract the videos url.
#get_url - will send the request to Scraper API so it can retrieve the HTML response.

#from what i've seen Scraper API doesn't work on instragram anymore so that's the above that may work better? ^

In [ ]:
#profile URL 
https://www.instagram.com/<user_name>/

## Tags URL
https://www.instagram.com/explore/tags/<example_tag>/

## Location URL
https://www.instagram.com/explore/locations/<location_id>/

# Note: the location URL is a numeric value so you need to identify the location ID number for
# the locations you want to scrape. 

In [ ]:
#retreiveing instragram accoutns 

In [ ]:
def start_requests(self):
        for username in user_accounts:
            url = f'https://www.instagram.com/{username}/?hl=en'
            yield scrapy.Request(get_url(url), callback=self.parse)

In [ ]:
#scraping posts JSON 

In [ ]:
def parse(self, response):
        x = response.xpath("//script[starts-with(.,'window._sharedData')]/text()").extract_first()
        json_string = x.strip().split('= ')[1][:-1]
        data = json.loads(json_string)

In [ ]:
def parse(self, response):
        x = response.xpath("//script[starts-with(.,'window._sharedData')]/text()").extract_first()
        json_string = x.strip().split('= ')[1][:-1]
        data = json.loads(json_string)
        # all that we have to do here is to parse the JSON we have
        user_id = data['entry_data']['ProfilePage'][0]['graphql']['user']['id']
        next_page_bool = \
            data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info'][
                'has_next_page']
        edges = data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_felix_video_timeline']['edges']
        for i in edges:
            url = 'https://www.instagram.com/p/' + i['node']['shortcode']
            video = i['node']['is_video']
            date_posted_timestamp = i['node']['taken_at_timestamp']
            date_posted_human = datetime.fromtimestamp(date_posted_timestamp).strftime("%d/%m/%Y %H:%M:%S")
            like_count = i['node']['edge_liked_by']['count'] if "edge_liked_by" in i['node'].keys() else ''
            comment_count = i['node']['edge_media_to_comment']['count'] if 'edge_media_to_comment' in i[
                'node'].keys() else ''
            captions = ""
            if i['node']['edge_media_to_caption']:
                for i2 in i['node']['edge_media_to_caption']['edges']:
                    captions += i2['node']['text'] + "\n"

            if video:
                image_url = i['node']['display_url']
            else:
                image_url = i['node']['thumbnail_resources'][-1]['src']
            item = {'postURL': url, 'isVideo': video, 'date_posted': date_posted_human,
                    'timestamp': date_posted_timestamp, 'likeCount': like_count, 'commentCount': comment_count, 'image_url': image_url,
                    'captions': captions[:-1]}

In [ ]:
#extracting video URL 

In [ ]:
if video:
     yield scrapy.Request(get_url(url), callback=self.get_video, meta={'item': item})
else:
     item['videoURL'] = ''
     yield item

In [ ]:
#get video

In [ ]:
def get_video(self, response):
        # only from the first page
        item = response.meta['item']
        video_url = response.xpath('//meta[@property="og:video"]/@content').extract_first()
        item['videoURL'] = video_url
        yield item

In [ ]:
#setting up proxies 
#API so not sure if it works 

In [ ]:
API = ‘<YOUR_API_KEY>’

def get_url(url):
    payload = {'api_key': API, 'url': url}
    proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
    return proxy_url


In [ ]:
def start_requests(self):
        for username in user_accounts:
            url = f'https://www.instagram.com/{username}/?hl=en'
            yield scrapy.Request(get_url(url), callback=self.parse)

In [ ]:
class InstagramSpider(scrapy.Spider):
    name = 'instagram'
    allowed_domains = ['api.scraperapi.com']
    custom_settings = {'CONCURRENT_REQUESTS_PER_DOMAIN': 5}


In [ ]:
#setting up monitoring 

In [ ]:
pip install scrapeops-scrapy

In [ ]:
## settings.py

## Add Your ScrapeOps API key
SCRAPEOPS_API_KEY = 'YOUR_API_KEY'

## Add In The ScrapeOps Extension
EXTENSIONS = {
 'scrapeops_scrapy.extension.ScrapeOpsMonitor': 500, 
}

## Update The Download Middlewares
DOWNLOADER_MIDDLEWARES = { 
'scrapeops_scrapy.middleware.retry.RetryMiddleware': 550, 
'scrapy.downloadermiddlewares.retry.RetryMiddleware': None, 
}

In [ ]:
#go live 

In [ ]:
scrapy crawl instagram -o test.csv